In [1]:
import os
import sys
import time
import boto3
import logging
import pandas as pd
import openmatrix as omx
from zipfile import ZipFile

s3_client = boto3.client('s3')

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO,
    format='%(asctime)s %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


os.chdir('../')
import kpi

In [14]:
def download_s3(local_file_name,s3_bucket,s3_object_key):
    """
    reference:
    https://stackoverflow.com/questions/41827963/
    track-download-progress-of-s3-file-using-boto3-and-callbacks
    """

    meta_data = s3_client.head_object(Bucket=s3_bucket, Key=s3_object_key)
    total_length = int(meta_data.get('ContentLength', 0))
    downloaded = 0

    def progress(chunk):
        nonlocal downloaded
        downloaded += chunk
        done = int(50 * downloaded / total_length)
        sys.stdout.write("\r[%s%s]" % ('=' * done, ' ' * (50-done)) )
        sys.stdout.flush()

    logger.info(f'Downloading {s3_object_key}')
    with open(local_file_name, 'wb') as f:
        s3_client.download_fileobj(s3_bucket, s3_object_key, f, Callback=progress)
        
def download_data(scenario):
    """
    Download results (ActivitySim) of scenario in a tmp folder. 
    
    Parameters: 
    -------------
    - scenario: str. scenario name 
    
    Returns:
    --------
    None
    
    """
    s3_bucket = 'carb-results'

    file_names = ['final_households.csv', 
                  'final_persons.csv', 
                  'final_trips.csv', 
                  'skims.omx']

    for file in file_names: 
        
        asim_local_file = "/Users/juandavidcaicedocastro/tmp/{}/{}".format(scenario, file)
        asim_s3_object_key = "{}/{}".format(scenario, file)
        file_exist = os.path.exists(asim_local_file)
        print(file_exist)

        if file_exist:
            print ('File {} already exists'.format(file))
            pass

        else:
            print ('Downloading {}'.format(file))
            download_s3(asim_local_file, s3_bucket , asim_s3_object_key)
            
    
    return None    

In [15]:
scenarios = os.listdir('/Users/juandavidcaicedocastro/tmp')
a = pd.Series(scenarios).sort_values()
b = a[~a.str.contains('fare')]
c = b[~b.str.contains('av_')]
d = c[~c.str.contains('.DS')]
scenarios = list(d)

In [16]:
# scenarios[22:31] : Done
# scenarios[2:22]: Done
# scenarios[-8:]: Done
# scenarios[2]

In [19]:
[scenarios[2]]

['03_transit_frequencies_+100']

In [ ]:
scenario
asim_local_file = "/Users/juandavidcaicedocastro/tmp/{}/{}".format(scenario, file)
asim_s3_object_key = "{}/{}".format(scenario, file)
file_exist = os.path.exists(asim_local_file)

In [20]:
for scenario in [scenarios[2]]:
    print ('Download Data for: {}'.format(scenario))
    download_data(scenario)
    dict_results = kpi.kpi_pilates(scenario)
    kpi.save_yaml('kpis_new/kpi_{}'.format(scenario), dict_results)
    print('')

Download Data for: 03_transit_frequencies_+100
True
File final_households.csv already exists
True
File final_persons.csv already exists
True
File final_trips.csv already exists
True
File skims.omx already exists
2022-10-22 11:31:14,417 root - INFO - Saving policy scenario resutls
Trips shape: (25623977, 14)
Households shape: (2877015, 60)
Mean TRIP VOT: 9.650615288034512
Trips shape, after merging: (25623977, 32)
Persons shape after merging: (7511523, 90)
Sum of c_cost: -85126.00611404733
Sum of c_ivt: -641718.232299998
Sum of value of time: 247287144.1764447
Sum of cs: -3411153350.741324
2022-10-22 11:37:10,052 root - INFO - Calulating total VMT...
2022-10-22 11:37:10,837 root - INFO - Calulating VMT per capita...
2022-10-22 11:37:10,878 root - INFO - Calulating VMT per capita by income...
2022-10-22 11:37:11,257 root - INFO - Calulating VMT per capita by race...
2022-10-22 11:37:13,314 root - INFO - Calulating VMT per capita by hispanic...
2022-10-22 11:37:15,500 root - INFO - Calula

## Debug 03_transit_frequencies

In [ ]:
scenario = '03_transit_frequencies_+100'
hh_fpath = os.path.join('/Users/juandavidcaicedocastro/tmp', scenario,'final_households.csv')
pp_fpath = os.path.join('/Users/juandavidcaicedocastro/tmp', scenario,'final_persons.csv')
tt_fpath = os.path.join('/Users/juandavidcaicedocastro/tmp', scenario,'final_trips.csv')
ss_fpath = os.path.join('/Users/juandavidcaicedocastro/tmp', scenario,'skims.omx')

In [ ]:
households = pd.read_csv(hh_fpath, index_col = 'household_id')
persons = pd.read_csv(pp_fpath, index_col = 'person_id')
trips = pd.read_csv(tt_fpath, index_col = 'trip_id', dtype = {'origin':int, 'destination':int})
skims = omx.open_file(ss_fpath, 'r')
mapping = read_yaml('/Users/juandavidcaicedocastro/Dropbox/01_berkeley/22_UrbanSim/github/sensitivy_analysis_carb/mapping.yaml')